# Understanding dplyr library in R
This exercise is to understand the usage of different functions in the dplyr library in R using a dataset for NYC Flights. This data set is also imported as a library into the script and then made use of later

In [35]:
library(dplyr)
library(nycflights13)

# Connecting to the Database
Connecting to a new database and creating if it does not exist. Then copying the flights table from the nycflights13 library into our newly created database using the copy_to function.
* The data that is inserted into the database is also stored in a variable named flights_sqlite

In [2]:
my_db <- src_sqlite("flights.sqlite", create = T)

In [3]:
flights_sqlite <- copy_to(my_db, flights, temporary = FALSE, indexes = list(c("year","month","day"), "carrier", "tailnum"), )

# Dispalying the Contents of the Flights Table

In [4]:
#Displaying the contents of the Flights Table
tbl(src = my_db, sql("Select * from flights"))

Source:   query [?? x 19]
Database: sqlite 3.11.1 [flights.sqlite]

    year month   day dep_time sched_dep_time dep_delay arr_time sched_arr_time
   <int> <int> <int>    <int>          <int>     <dbl>    <int>          <int>
1   2013     1     1      517            515         2      830            819
2   2013     1     1      533            529         4      850            830
3   2013     1     1      542            540         2      923            850
4   2013     1     1      544            545        -1     1004           1022
5   2013     1     1      554            600        -6      812            837
6   2013     1     1      554            558        -4      740            728
7   2013     1     1      555            600        -5      913            854
8   2013     1     1      557            600        -3      709            723
9   2013     1     1      557            600        -3      838            846
10  2013     1     1      558            600        -2      753

# Displaying selected columns for all the flights
This is done using the select function in dplyr. For this example, I am displaying the year, month, date, departure delay and the arrival delay for all the flights.

In [5]:
select(flights_sqlite, year:day, dep_delay, arr_delay)

Source:   query [?? x 5]
Database: sqlite 3.11.1 [flights.sqlite]

    year month   day dep_delay arr_delay
   <int> <int> <int>     <dbl>     <dbl>
1   2013     1     1         2        11
2   2013     1     1         4        20
3   2013     1     1         2        33
4   2013     1     1        -1       -18
5   2013     1     1        -6       -25
6   2013     1     1        -4        12
7   2013     1     1        -5        19
8   2013     1     1        -3       -14
9   2013     1     1        -3        -8
10  2013     1     1        -2         8
# ... with more rows

# Filtering out Flights
Using the filter function, I am selecting all the flights that have a departure delay of more than 240. This function is also available in the dplyr library.

In [6]:
filter(flights_sqlite, dep_delay > 240)

Source:   query [?? x 19]
Database: sqlite 3.11.1 [flights.sqlite]

    year month   day dep_time sched_dep_time dep_delay arr_time sched_arr_time
   <int> <int> <int>    <int>          <int>     <dbl>    <int>          <int>
1   2013     1     1      848           1835       853     1001           1950
2   2013     1     1     1815           1325       290     2120           1542
3   2013     1     1     1842           1422       260     1958           1535
4   2013     1     1     2115           1700       255     2330           1920
5   2013     1     1     2205           1720       285       46           2040
6   2013     1     1     2343           1724       379      314           1938
7   2013     1     2     1332            904       268     1616           1128
8   2013     1     2     1412            838       334     1710           1147
9   2013     1     2     1607           1030       337     2003           1355
10  2013     1     2     2131           1512       379     2340

# Sorting Data
Sorting the data is done using the arrange function. In this example, I am sorting them by year, then month in case of tie and then the day in case of a 2nd tie. It will sort them in a non-decreasing way which can be changed to decreasing by using the desc function on any parameter.

In [7]:
arrange(flights_sqlite, year, month, day)

Source:   query [?? x 19]
Database: sqlite 3.11.1 [flights.sqlite]

    year month   day dep_time sched_dep_time dep_delay arr_time sched_arr_time
   <int> <int> <int>    <int>          <int>     <dbl>    <int>          <int>
1   2013     1     1      517            515         2      830            819
2   2013     1     1      533            529         4      850            830
3   2013     1     1      542            540         2      923            850
4   2013     1     1      544            545        -1     1004           1022
5   2013     1     1      554            600        -6      812            837
6   2013     1     1      554            558        -4      740            728
7   2013     1     1      555            600        -5      913            854
8   2013     1     1      557            600        -3      709            723
9   2013     1     1      557            600        -3      838            846
10  2013     1     1      558            600        -2      753

## Sorting all data in descending order
Soring all the data in descending order with the maximum departure delay first and then by the year, month and day if conflict arises.

In [8]:
arrange(flights_sqlite, desc(dep_delay), year, month, day)

Source:   query [?? x 19]
Database: sqlite 3.11.1 [flights.sqlite]

    year month   day dep_time sched_dep_time dep_delay arr_time sched_arr_time
   <int> <int> <int>    <int>          <int>     <dbl>    <int>          <int>
1   2013     1     9      641            900      1301     1242           1530
2   2013     6    15     1432           1935      1137     1607           2120
3   2013     1    10     1121           1635      1126     1239           1810
4   2013     9    20     1139           1845      1014     1457           2210
5   2013     7    22      845           1600      1005     1044           1815
6   2013     4    10     1100           1900       960     1342           2211
7   2013     3    17     2321            810       911      135           1020
8   2013     6    27      959           1900       899     1236           2226
9   2013     7    22     2257            759       898      121           1026
10  2013    12     5      756           1700       896     1058

# Generating More Data
Generating more data (meta-data) from already existing data is done using the mutate function in dplyr. We can simply refer to existing colums and create new columns by operating on those existing colums. In this example, I have calculated the speed of the aircraft using the distance and the air time columns in the same table.

After that, I have displayed the speed using the select function, as described above.

In [9]:
updated_flights <- mutate(flights_sqlite, speed=distance/air_time)

In [10]:
select(updated_flights, speed)

Source:   query [?? x 1]
Database: sqlite 3.11.1 [flights.sqlite]

      speed
      <dbl>
1  6.167401
2  6.237885
3  6.806250
4  8.612022
5  6.568966
6  4.793333
7  6.740506
8  4.320755
9  6.742857
10 5.311594
# ... with more rows

# Summary of Data
Summarising data to get some results is another great aspect of dplyr library. In the example below, I have calculated the average speed and the average air time of all the flights in the database and displayed them.

In [11]:
summarise(updated_flights, avg_speed = mean(speed), avg_airtime = mean(air_time))

Source:   query [?? x 2]
Database: sqlite 3.11.1 [flights.sqlite]

  avg_speed avg_airtime
      <dbl>       <dbl>
1  6.571228    150.6865

# Understanding Laziness of dplyr
dplr is a very lazy library and does not touch the database unless it is forced to. For example, as shown below, it will not manipulate the data on the database and will just display the top 10 results that is has in memory. Whenever we query it to display the date, it will just show the top 10 results and mention more results are available. 

In [12]:
c1 <- filter(flights_sqlite, year == 2013, month == 1, day == 1)
c2 <- select(c1, year, month, day, carrier, dep_delay, air_time, distance)
c3 <- mutate(c2, speed = distance / air_time * 60)
c4 <- arrange(c3, desc(speed), year, month, day, carrier)

In [13]:
c4

Source:   query [?? x 8]
Database: sqlite 3.11.1 [flights.sqlite]

    year month   day carrier dep_delay air_time distance    speed
   <int> <int> <int>   <chr>     <dbl>    <dbl>    <dbl>    <dbl>
1   2013     1     1      AA        59      184     1623 529.2391
2   2013     1     1      B6         0      182     1598 526.8132
3   2013     1     1      DL         0      182     1598 526.8132
4   2013     1     1      B6        91      183     1598 523.9344
5   2013     1     1      UA        -5      189     1634 518.7302
6   2013     1     1      B6        -6      185     1598 518.2703
7   2013     1     1      B6        -1      183     1576 516.7213
8   2013     1     1      UA         3      187     1608 515.9358
9   2013     1     1      UA         1      188     1608 513.1915
10  2013     1     1      AA        -5      188     1598 510.0000
# ... with more rows

## Forcing dplyr to fetch all the data
Using the collect function, we can force the library to fetch the entire data and show as a table/data frame. When we execute this function, that is when the library actually fetches the data from the database and displays to us.

In [14]:
collect(c4)

year,month,day,carrier,dep_delay,air_time,distance,speed
2013,1,1,AA,59,184,1623,529.2391
2013,1,1,B6,0,182,1598,526.8132
2013,1,1,DL,0,182,1598,526.8132
2013,1,1,B6,91,183,1598,523.9344
2013,1,1,UA,-5,189,1634,518.7302
2013,1,1,B6,-6,185,1598,518.2703
2013,1,1,B6,-1,183,1576,516.7213
2013,1,1,UA,3,187,1608,515.9358
2013,1,1,UA,1,188,1608,513.1915
2013,1,1,AA,-5,188,1598,510.0000


## Understanding the Operations performed by dplyr
Since we do not deal with SQL directly when using dplyr, we don't really know what query is dplyr sending to the database to fetch our results. We can get this information when we use the explain function in dplyr. This function displays the entire SQL command that it executes on the database to fetch our results. 
In the example below, I am fetching the SQL commands for the query c4 that I built above.

In [15]:
explain(c4)

<SQL>
SELECT *
FROM (SELECT `year`, `month`, `day`, `carrier`, `dep_delay`, `air_time`, `distance`, `distance` / `air_time` * 60.0 AS `speed`
FROM (SELECT `year` AS `year`, `month` AS `month`, `day` AS `day`, `carrier` AS `carrier`, `dep_delay` AS `dep_delay`, `air_time` AS `air_time`, `distance` AS `distance`
FROM (SELECT *
FROM `flights`
WHERE ((`year` = 2013.0) AND (`month` = 1.0) AND (`day` = 1.0)))))
ORDER BY `speed` DESC, `year`, `month`, `day`, `carrier`


<PLAN>
   addr       opcode p1    p2 p3              p4 p5 comment
1     0         Init  0    47  0                 00      NA
2     1   SorterOpen  4    14  0 k(5,-B,B,B,B,B) 00      NA
3     2     OpenRead  3     2  0              19 00      NA
4     3     OpenRead  5 22527  0        k(4,,,,) 02      NA
5     4         Real  0     1  0            2013 00      NA
6     5         Real  0     2  0               1 00      NA
7     6         Real  0     3  0               1 00      NA
8     7     Affinity  1     3  0             